# Method 1

In [ ]:
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sentence_transformers import SentenceTransformer
import numpy as np
import csv

In [ ]:
model_mpnet = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
train = pd.read_csv('train_yelp_60k.csv')
test = pd.read_csv('test_yelp_60k.csv')

In [ ]:
# Drop ID from test and train dataframe
train_df = train.drop('ID', 1)
test_df = test.drop('ID', 1)

<ipython-input-4-2b974a0e1338>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  train_df = train.drop('ID', 1)
<ipython-input-4-2b974a0e1338>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  test_df = test.drop('ID', 1)


In [ ]:
text_embedding_train = model_mpnet.encode(train_df['Text'])
text_embedding_test = model_mpnet.encode(test_df['Text'])

In [ ]:
X_train = pd.DataFrame(text_embedding_train)
X_test = pd.DataFrame(text_embedding_test)

In [ ]:
y = train_df['Class']

In [ ]:
linearModel = LogisticRegression(max_iter=1000)
linearModel.fit(X_train, y)
y_pred = linearModel.predict(X_test)

In [ ]:
y_pred

array(['negative', 'positive', 'positive', ..., 'positive', 'positive',
       'positive'], dtype=object)

In [ ]:
# test = pd.read_csv('test_yelp_60k.csv')
# ids = test['ID'].to_numpy()
# data = np.column_stack((ids, y_pred))

# csv_file_path = 'prediction2.csv'

# # Write to CSV file
# with open(csv_file_path, 'w', newline='') as csvfile:
#     csv_writer = csv.writer(csvfile)
#     csv_writer.writerow(['ID', 'CLASS'])
#     csv_writer.writerows(data)

# Method 2

In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split

## Reading the CSV file
import pandas as pd
df = pd.read_csv('train_yelp_60k.csv')
df.head()

## Dropping the "ID" column from the dataframe,
df = df.drop('ID', axis=1)


## Splitting the dataset into Dependent and Independent Variable
## Putting into list so that they can be processed
from sklearn.model_selection import train_test_split
X = list(df['Text'])
y = list(df['Class'])

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(X, y, test_size=0.2, random_state=42)

# BERT Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenizing data
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

# Mapping the values
label2id = {'positive': 0, 'neutral': 1, 'negative': 2}
train_labels = [label2id[label] for label in train_labels]
val_labels = [label2id[label] for label in val_labels]

# Coverting to TensorFlow datatype to make it compatible for the classifier
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

In [ ]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)  # 3 possible outputs: positive, negative, or neutral.

## We prepare the dataset to to be used for the classifier
## We do this batching because the google collab file was crashing without
## sufficient RAM. So, we switched to T4 GPU andd batch size to 4

train_dataset = train_dataset.shuffle(len(train_texts)).batch(4)
val_dataset = val_dataset.batch(4)

## We took a small learning rate to tackle
## Overfitting. It look longer than normal to finish because of that.
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

# Train the model
mainModel = model.fit(train_dataset, epochs=1, validation_data=val_dataset)


In [ ]:
test_df = pd.read_csv('test_yelp_60k.csv')

test_texts = test_df['Text'].tolist()

# Tokenizing the test data
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# Coverting to TensorFlow datatype to make it compatible for the classifier
test_labels = [0] * len(test_texts)  ## Making a placeholder for test labels
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
)).batch(4)  # Batch size is 4 because higher was crashing on Google Collab

# Make predictions on the test dataset
predictions = model.predict(test_dataset)

# Get predicted labels from model output logits
predicted_classes = tf.argmax(predictions.logits, axis=1).numpy()

In [ ]:
## Mapping the values back from numerical to alphabetical
id2label = {v: k for k, v in label2id.items()}
predicted_labels = [id2label[label_id] for label_id in predicted_classes]

# Add Class to the test DataFrame
test_df['Class'] = predicted_labels

# Creation of CSV File
test_df.to_csv('prediction2.csv', index=False)

In [ ]:
df2 = pd.read_csv("prediction2.csv")

In [ ]:
df2["Class"].value_counts()